<VSCode.Cell language="markdown">
# Uvicorn / FastAPI アプリの NameError(llm_client) 調査ノートブック

このノートブックでは、`uvicorn chat_app:app` 実行時に発生する `NameError: llm_client is not defined` の原因を特定し、
`llm_client` の定義と FastAPI アプリへの注入方法を確認・修正する流れをまとめます。
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 1. 環境と依存パッケージの確認

このセクションでは、Python バージョン・仮想環境(venv)・必要なパッケージが正しくインストールされているかを確認します。
</VSCode.Cell>

In [ ]:
import sys, subprocess
import platform

print("Python version:", sys.version)
print("Platform:", platform.platform())

# venv 上の pip で主要パッケージを確認
subprocess.run([sys.executable, "-m", "pip", "list"], check=False)

# 必要に応じてパッケージをインストール（コメントアウトを外して使う）
# required = ["uvicorn", "fastapi", "openai"]  # Azure OpenAI SDK を使うなら "openai" など
# subprocess.run([sys.executable, "-m", "pip", "install", *required], check=False)

<VSCode.Cell language="markdown">
## 2. chat_app.py のエントリーポイントと設定の確認

`uvicorn chat_app:app` が参照する `app` オブジェクトや `llm_client` の使われ方を確認します。
</VSCode.Cell>

In [ ]:
from pathlib import Path

chat_app_path = Path("chat_app.py")
print("chat_app.py exists:", chat_app_path.exists(), "at", chat_app_path.resolve())

if chat_app_path.exists():
    code = chat_app_path.read_text(encoding="utf-8")
    # app 定義付近と llm_client の記述を簡易的に検索
    for kw in ["app =", "FastAPI", "llm_client", "azure_openai_client"]:
        print(f"\n===== Snippet around '{kw}' =====")
        idx = code.find(kw)
        if idx != -1:
            start = max(0, idx - 200)
            end = min(len(code), idx + 200)
            print(code[start:end])
else:
    print("chat_app.py が見つかりません。ノートブックのカレントディレクトリを確認してください。")

<VSCode.Cell language="markdown">
## 3. llm_client の生成コードを実装

Azure OpenAI のエンドポイントやキー、デプロイ名を環境変数から読み込み、`llm_client` を生成するコードを確認・試作します。
</VSCode.Cell>

In [ ]:
import os

from typing import Optional

try:
    from openai import AzureOpenAI
except ImportError:
    AzureOpenAI = None
    print("openai パッケージが未インストールです。必要であれば `pip install openai` を実行してください。")

AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION", "2024-05-01-preview")

print("AZURE_OPENAI_ENDPOINT =", AZURE_OPENAI_ENDPOINT)
print("AZURE_OPENAI_API_KEY set =", bool(AZURE_OPENAI_API_KEY))
print("AZURE_OPENAI_API_VERSION =", AZURE_OPENAI_API_VERSION)

llm_client: Optional[object] = None
if AzureOpenAI is not None and AZURE_OPENAI_ENDPOINT and AZURE_OPENAI_API_KEY:
    llm_client = AzureOpenAI(
        api_key=AZURE_OPENAI_API_KEY,
        api_version=AZURE_OPENAI_API_VERSION,
        azure_endpoint=AZURE_OPENAI_ENDPOINT,
    )
    print("llm_client 初期化成功:", llm_client)
else:
    print("llm_client を初期化できませんでした。環境変数と openai パッケージを確認してください。")

<VSCode.Cell language="markdown">
## 4. FastAPI アプリへの llm_client 注入と DI の確認

`chat_app.py` 内で `llm_client` をどのように FastAPI アプリへ渡すかを確認し、足りない場合の DI パターン例を示します。
</VSCode.Cell>